# Entrega laboratorio 5

## Integrantes 

- Javier Steven Barrera Toro - 202214779
- Julian Santiago Rolon Toloza - 202215839

In [2]:
from matplotlib import pyplot as plt
from typing import List, Tuple, NewType
import numpy as np
import pandas as pd
import sympy as sp
import copy
import tabulate

np.seterr(divide='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

# Problema 1

## 1.1 Formulación del modelo multiobjetivo

### 1.1.1 Conjuntos

Definimos 4 conjuntos: 

- $R$: Conjunto de recursos {Alimentos, Medicinas, Equipos, Agua, Mantas}
- $A$: Conjunto de aviones {1,2,3,4}
- $Z$: COnjunto de ZOnas de destino {A,B,C,D}
- $V$: Conjunto de viajes posibles por avión {1,2}

### 1.1.2 Parámetros

#### Parametros para los recursos: 
- $v_i$: Valor de impacto por tonelada del recurso $i$
- $p_i$: Peso (TON) por unidad del recurso $i$
- $u_i$: Volumen $(m^3)$ por unidad del recurso $i$
- $d_i$: Disponibilidad total (unidades) del recurso $i$

#### Aviones: 
- $capW_j$: Capacidad del peso (TON) del avión $j$
- $capV_j$: Capacidad del volumen $(m^3)$ del avion $j$
- $CF_j$: COsto fijo por uso del avión $j$ (miles USD)
- $CV_j$: Costo variable por jm del avión $j$ (miles USD/km)

#### Zonas:
- $dist_k$: Distancia a la zona $k$ (km)
- $mult_k$: multiplicador de impacto de la zona $k$
- $min_{ik}$: Necesidad mínima del reecurso $i$ en la zona $k$ (TON)

### 1.1.3 Variables de decisión

- $x_{ijvk}$: Cantidad (TON) del recurso $i$ transportado por el avion $j$en viaje $v$ a la zona $k$
- $y_jvk /in {(0,1)}: 1$ si el avion $j$ en el viaje $v$ va a la zona $k$, 0 en caso contrario
- 

### 1.1.4 Funciones objetivo

### 1.1.5 Restricciones

## 1.2 Implementación del Método

## 1.3 Análisis y Descución

# Problema 2

## Formulación del modelo multiobjetivo

### Conjuntos

Se define el conjunto de localidades $N$ que se tienen que inspeccionar y evaluar. Además, el conjunto $E$ con todas las rutas posibles entre las localidades que se deben inspeccionar.

### Parámetros

La matriz $d_{ij}$ tiene la distancia del lugar $i\in N$ al lugar $j \in N \,|\, i \neq j$, ya que no existen autociclos. Cada ruta tiene un riesgo asociado $r_{ij}$ y cada $i \in N$ localidad tiene una calidad de inspección $n_i$. También se define un nodo $o \in N$ que será el punto de partida para que el equipo haga la inspección. 

### Variables de decisión

La variable de decisión es $X_{ij}$ que indica si el camino desde la localidad $i$ a $j$ se selecciona. El dominio de la variable de decisión es binaria ($X_{ij} \in \{0, 1\}$), 1 si se selecciona y 0 de lo contrario. Además, es necesario incluir una variable auxiliar $u_i$ para cada $i \in N \backslash \{o\}$ para darle un orden a las localidades recorridas, por ejemplo, si se toma el camino $(i,j) \in E$ el valor de $u_j > u_i$. Asimismo, los valores que puede tomar $u_i$ son, ya que se sabe que el nodo origen es 0, de $1$ hasta $|N|$. 

### Funciones objetivo

Bien se sabe que se quieren optimizar tres aspectos de este problema. Se tiene que minimizar distancia recorrida. 
$$
Z_1 = \min \sum_{(i,j) \in E} X_{ij} \times d_{ij}
$$
Además, se debe máximizar la calidad de inspección de la ruta. 
$$
Z_2 = \max \sum_{(i,j) \in E} X_{ij} \times n_i \times d_{ij}
$$
También se debe minimizar el riesgo de las rutas elegidas. 
$$
Z_3 = \min \sum_{(i,j) \in E} X_{ij} \times r_{ij}
$$

Recordemos que para poder realizar la optimización todas las funciones deben o maximizar o minimizar. Por esto, es necesario cambiar el objetivo de $Z_2$, en este caso $Z_2^*=\min-Z_2$.
$$
Z_2^* = \min \left(-\sum_{(i,j) \in E} X_{ij} \times n_i \times d_{ij} \right)
$$

Finalmente, la función multiobjetivo es 
$$
\min F = (Z_1, Z_2^*, Z_3)^T
$$

### Restricciones

Se sabe que el equipo debe salir del nodo origen.
$$
\sum_{j \in N} X_{ij} = 1, \forall i \in N \, | \, i = o
$$
El equipo de inspección debe volver al nodo de origen. 
$$
\sum_{i \in N} X_{ij} = 1, \forall j \in N \, | \, j = o
$$
El equipo debe entrar a cada una de las localidades. 
$$
\sum_{i \in N \, | \, i \neq j} X_{ij} = 1, \forall j \in N \backslash \{o\}
$$
El equipo debe salir de cada una de las localidades. 
$$
\sum_{j \in N \, | \, j \neq i} X_{ij} = 1, \forall i \in N \backslash \{o\}
$$
Adicionalmente, se debe agregar la restricción MTZ para la eliminación de subtoures.
$$
u_i - u_j + (|N| - 1) X_{ij} \leq |N| - 2, \forall i, j \in N \, | \, 1 \leq i \wedge j \leq |N| \wedge i \neq j
$$

## Selección y justificación del método de resolución

En un primer momento se habia considerado utilizar $\epsilon$-constraint dado que no se conoce si el frente de pareto óptimo tiene secciones no convexas, sin embargo, dado que se desean minimizar todos los objetivos sin darle más importancia a uno que a otro se opta por usar **suma ponderada**. Además, parece ser una buena opción hallar una solución equilibrada entre los distintos objetivos dado el contexto del problema ya que no se le estaría dando más prioridad a un objetivo y restringiendo los demás. Además, dada la naturaleza de **depende** del contexto del problema se le puede dar más o menos peso a un objetivo dependiendo de si, por ejemplo, modelo una zona en la que hay bastantes robos. 

## Implementación y análisis

## Análisis y discusión adicional